# libraries connections and keys

In [2]:
#importing libraries

import pandas as pd
import datetime
from datetime import date, timedelta
import gspread
import pygsheets
from oauth2client.service_account import ServiceAccountCredentials
import warnings
warnings.filterwarnings("ignore")

#Columns to add additionally

week_end_date=pd.DataFrame()
week_end_date['week end date']=''
city=pd.DataFrame()
city['City']='Mumbai'

#creating connections for gspread and pygsheets

scope= ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials= ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope) 
client= gspread.authorize(credentials)

clients = pygsheets.authorize(service_file='client_secret.json')


#sheet keys

car_master='1_r5OMN1P8Tof5IRaE5jYd-jHaP8j15avQVMycDJdhec'
leasing_master='1D0U02U0QN7pXsgM4AdOUiUKB4rB-SoTwSspWkuwl_8E'
chirag_sir_adjustment_summary='14bR0MmfLvQi1lyzpIgFn02Qizr9bLwIDqhBu1vqxk2E'
leasing_vinay_bhai='1gorOufgYoOg7akqGWZtSsHU4Se92WUv0-kMzEw19Zh4'
car_master_vinay_bhai='1KbgIgaVhBUyJa4EAEgJpr42_qcLmO371qUzA8EEEbmU'
pune_car_master='1S9qECmC9EEo4e62Ujby9ATYUjkJbYE36BuvLtJFs_9c'
hyd_car_master='1LOWIFyOpU5psUP3wq-RA0H-1Er6fqkZM2IZ0i910Z7c'
hyd_leasing='1OQfyvkrySTxWzTbGBG1oTm0oqhvAVa7Y8dUaobLsH5Y'
bangalore='1_7Wvxkz5r3baEUV_WMCcrBnzNE9-Y74Yb6S0VVn7PEw'
delhi='1CccQrtO-_DGzpnUK9trmHqxLRzQyxkmcWEdLkZ5HBss'

ola_data_sheet='17qqSTzHV6yuerajrlobyqUPH8d9z54XqM5MYQP2Sxtw'

# #Adjustments ( chirag sir sheet going to reject some type from type column)

# sheet= client.open_by_key(car_master)
# ws= sheet.worksheet('Adjustments')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# adjustment_df=dfm.iloc[:,[0,3,5]]
# array=adjustment_df['Type'].unique()
# df=pd.DataFrame(array,columns=['Type'])
# sheet= client.open_by_key(chirag_sir_adjustment_summary)
# ws= sheet.worksheet('Adjustment')
# ws.update([df.columns.values.tolist()] + df.values.tolist(),value_input_option='USER_ENTERED')

# mumbai car master and leasings-korum mall

In [9]:
##################################################car master##########################################################

#uber

sheet= clients.open_by_key(car_master)
ws= sheet.worksheet_by_title('Uber')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
uber_df=dfm.iloc[:,[0,15,21,23]]
uber_df['Week start Date'] = pd.to_datetime(uber_df['Week start Date'], errors='coerce',format='%d/%m/%Y')
uber_df['Tolls']=uber_df['Tolls'].str.replace(',','')
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].str.replace(',','')
uber_df['Incentive'] = uber_df['Incentive'].str.replace(',','')
uber_df['Tolls'] = pd.to_numeric(uber_df['Tolls'], errors = 'coerce')
uber_df['60%+Tolls + Tips'] = pd.to_numeric(uber_df['60%+Tolls + Tips'], errors = 'coerce')
uber_df['Incentive'] = pd.to_numeric(uber_df['Incentive'], errors = 'coerce')
uber_df.rename({'Tolls': 'car-master-Uber-toll', '60%+Tolls + Tips': 'car-master-Uber-60%+Tolls + Tips','Incentive':'car-master-Uber-Incentive'}, axis=1, inplace=True)
uber = uber_df.groupby(pd.Grouper(freq='W', key='Week start Date'))['car-master-Uber-toll','car-master-Uber-60%+Tolls + Tips','car-master-Uber-Incentive'].sum()
uber.index -= pd.Timedelta(days=6)

#b2b

sheet= clients.open_by_key(car_master)
ws= sheet.worksheet_by_title('B2B')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
b2b_df=dfm.iloc[:,[1,9,10,12]]
b2b_df['Date'] = pd.to_datetime(b2b_df['Date'], errors='coerce',format='%d-%m-%Y')
b2b_df['Fuel']=b2b_df['Fuel'].str.replace(',','')
b2b_df['Toll'] = b2b_df['Toll'].str.replace(',','').str.strip()
b2b_df['TotalAmount'] = b2b_df['TotalAmount'].str.replace(',','')
b2b_df['Fuel'] = pd.to_numeric(b2b_df['Fuel'], errors = 'coerce')
b2b_df['TotalAmount'] = pd.to_numeric(b2b_df['TotalAmount'], errors = 'coerce')
b2b_df['Toll'] = pd.to_numeric(b2b_df['Toll'], errors = 'coerce')
b2b_df.rename({'Fuel': 'car-master-B2B-Fuel', 'Toll': 'car-master-B2B-Toll','TotalAmount':'car-master-B2B-TotalAmount'}, axis=1, inplace=True)
b2b = b2b_df.groupby(pd.Grouper(freq='W', key='Date'))['car-master-B2B-Fuel','car-master-B2B-Toll','car-master-B2B-TotalAmount'].sum()
b2b.index -= pd.Timedelta(days=6)

#Penalty

sheet= clients.open_by_key(car_master)
ws= sheet.worksheet_by_title('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('-','/')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d/%m/%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = pd.to_numeric(Penalty_df['Fine Amount'], errors = 'coerce')
Penalty_df.rename({'Fine Amount': 'car-master-Penalty-Fine Amount'}, axis=1, inplace=True)
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['car-master-Penalty-Fine Amount'].sum()
df.index -= pd.Timedelta(days=6)
Penalty=pd.DataFrame(df)

#Adjustments

sheet= clients.open_by_key(chirag_sir_adjustment_summary)# from chirag sir adjustments to gather where type is not rejected
ws= sheet.worksheet_by_title('Adjustment')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,1]]
reject_df=df.loc[df['Rejection'] != 'reject']
unique_type=reject_df.iloc[:,[0]]

sheet= clients.open_by_key(car_master)#Adjustments - take selected type in selected column
ws= sheet.worksheet_by_title('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,3,5]]
adjustment_df=df[df["Type"].isin(unique_type["Type"])]

dfm=adjustment_df
dfm['Date'] = dfm['Date'].str.replace('/','-')
dfm['Date'] = pd.to_datetime(dfm['Date'], errors='coerce',format='%d-%m-%Y')
dfm['Amount'] = dfm['Amount'].str.replace(',','')
dfm['Amount'] = pd.to_numeric(dfm['Amount'], errors = 'coerce')
dfm.rename({'Amount': 'car-master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = dfm.groupby(pd.Grouper(freq='W', key='Date'))['car-master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment=pd.DataFrame(dfs)

#RTO

sheet= clients.open_by_key(car_master)
ws= sheet.worksheet_by_title('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[1,6]]
rto_df['Date in Payout'] = rto_df['Date in Payout'].str.replace('/','-')
rto_df['Date in Payout'] = pd.to_datetime(rto_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
rto_df.rename({'Fine Amount': 'car-master-RTO-Fine Amount'}, axis=1, inplace=True)
dfs = rto_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['car-master-RTO-Fine Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
rto=pd.DataFrame(dfs)

################################################################New Leasing Master##########################################

#Adjustments

sheet= clients.open_by_key(leasing_master)
ws= sheet.worksheet_by_title('Adjustment')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
adjustment_df=dfm.iloc[:,[0,10]]
adjustment_df['Date'] = adjustment_df['Date'].str.replace('-','/')
adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d/%m/%Y')
adjustment_df['Amount'] = adjustment_df['Amount'].str.replace(',','')
adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
adjustment_df.rename({'Amount': 'New-Leasing-Master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = adjustment_df.groupby(pd.Grouper(freq='W', key='Date'))['New-Leasing-Master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment_1=pd.DataFrame(dfs)

#RTO

sheet= clients.open_by_key(leasing_master)
ws= sheet.worksheet_by_title('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[0,6]]
rto_df['Date of Fine'] = rto_df['Date of Fine'].str.replace('-','/')
rto_df['Date of Fine'] = pd.to_datetime(rto_df['Date of Fine'], errors='coerce',format='%d/%m/%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
rto_df.rename({'Fine Amount': 'New-Leasing-Master-RTO-Fine Amount'}, axis=1, inplace=True)
dfs = rto_df.groupby(pd.Grouper(freq='W', key='Date of Fine'))['New-Leasing-Master-RTO-Fine Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
rto_1=pd.DataFrame(dfs)

#Penalty

sheet= clients.open_by_key(leasing_master)
ws= sheet.worksheet_by_title('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = pd.to_numeric(Penalty_df['Fine Amount'], errors = 'coerce')
Penalty_df.rename({'Fine Amount': 'New-Leasing-Master-Penalty-Fine Amount'}, axis=1, inplace=True)
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['New-Leasing-Master-Penalty-Fine Amount'].sum()
df.index -= pd.Timedelta(days=6)
Penalty_1=pd.DataFrame(df)

#ola

sheet= clients.open_by_key(ola_data_sheet)
ws= sheet.worksheet_by_title('Ola')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
ola_df=dfm.iloc[:,[0,5,6]]
ola_df['Date'] = ola_df['Date'].str.replace('-','/')
ola_df['Date'] = pd.to_datetime(ola_df['Date'], errors='coerce',format='%d/%m/%Y')
ola_df['Operator Bill Raw'] = ola_df['Operator Bill Raw'].str.replace(',','')
ola_df['Toll/ Parking for Hisaab'] = ola_df['Toll/ Parking for Hisaab'].str.replace(',','')
ola_df['Operator Bill Raw'] = pd.to_numeric(ola_df['Operator Bill Raw'], errors = 'coerce')
ola_df['Toll/ Parking for Hisaab'] = pd.to_numeric(ola_df['Toll/ Parking for Hisaab'], errors = 'coerce')
ola_df.rename({'Operator Bill Raw':'New-Leasing-Master-ola-Operator Bill Raw','Toll/ Parking for Hisaab':'New-Leasing-Master-ola-Toll/ Parking for Hisaab'}, axis=1, inplace=True)
dfs = ola_df.groupby(pd.Grouper(freq='W', key='Date'))['New-Leasing-Master-ola-Operator Bill Raw','New-Leasing-Master-ola-Toll/ Parking for Hisaab'].sum()
dfs.index -= pd.Timedelta(days=6)
ola_1=pd.DataFrame(dfs)

#weekly sheet

sheet= clients.open_by_key(leasing_master)
ws= sheet.worksheet_by_title('Weekly Sheet')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
weekly_df=dfm.iloc[:,[0,7,13,18,22]]
weekly_df['Start Date'] = weekly_df['Start Date'].str.replace('-','/')
weekly_df['Start Date'] = pd.to_datetime(weekly_df['Start Date'], errors='coerce',format='%d/%m/%Y')
weekly_df['Rental Amount'] = weekly_df['Rental Amount'].str.replace(',','')
weekly_df['Uber Rev 1'] = weekly_df['Uber Rev 1'].str.replace(',','')
weekly_df['Uber Rev 2'] = weekly_df['Uber Rev 2'].str.replace(',','')
weekly_df['Uber Incentive'] = weekly_df['Uber Incentive'].str.replace(',','')
weekly_df['Rental Amount'] = pd.to_numeric(weekly_df['Rental Amount'], errors = 'coerce')
weekly_df['Uber Rev 1'] = pd.to_numeric(weekly_df['Uber Rev 1'], errors = 'coerce')
weekly_df['Uber Rev 2'] = pd.to_numeric(weekly_df['Uber Rev 2'], errors = 'coerce')
weekly_df['Uber Incentive'] = pd.to_numeric(weekly_df['Uber Incentive'], errors = 'coerce')
weekly_df.rename({'Rental Amount':'New-Leasing-Master-weekly-Rental Amount','Uber Rev 1':'New-Leasing-Master-weekly-Uber Rev 1','Uber Rev 2':'New-Leasing-Master-weekly-Uber Rev 2','Uber Incentive':'New-Leasing-Master-weekly-Uber Incentive'}, axis=1, inplace=True)
weekly = weekly_df.groupby(pd.Grouper(freq='W', key='Start Date'))['New-Leasing-Master-weekly-Rental Amount','New-Leasing-Master-weekly-Uber Rev 1','New-Leasing-Master-weekly-Uber Rev 2','New-Leasing-Master-weekly-Uber Incentive'].sum()
weekly.index -= pd.Timedelta(days=6)
weekly_1=pd.DataFrame(weekly)

#summary
summary=pd.concat([week_end_date,city,uber,b2b,Penalty,adjustment,rto,adjustment_1,rto_1,Penalty_1,ola_1,weekly_1], axis=1)
summary.fillna('', inplace=True)
summary.index.name='week start date'
summary_df=summary.reset_index()
sums=summary_df.sort_values(by='week start date')

#pushing
sheet= clients.open_by_key(chirag_sir_adjustment_summary)
ws= sheet.worksheet_by_title('Mumbai Thane Summary')
ws.clear()
ws.set_dataframe(sums,(1,1))
print("Mumbai Thane Summary updated")

Mumbai Thane Summary updated


In [8]:
# ola_data_sheet='17qqSTzHV6yuerajrlobyqUPH8d9z54XqM5MYQP2Sxtw'
# sheet= clients.open_by_key(ola_data_sheet)
# ws= sheet.worksheet_by_title('Ola')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# ola_df=dfm.iloc[:,[0,5,6]]
# ola_df['Date'] = ola_df['Date'].str.replace('-','/')
# ola_df['Date'] = pd.to_datetime(ola_df['Date'], errors='coerce',format='%d/%m/%Y')
# ola_df['Operator Bill Raw'] = ola_df['Operator Bill Raw'].str.replace(',','')
# ola_df['Toll/ Parking for Hisaab'] = ola_df['Toll/ Parking for Hisaab'].str.replace(',','')
# ola_df['Operator Bill Raw'] = pd.to_numeric(ola_df['Operator Bill Raw'], errors = 'coerce')
# ola_df['Toll/ Parking for Hisaab'] = pd.to_numeric(ola_df['Toll/ Parking for Hisaab'], errors = 'coerce')
# ola_df.rename({'Operator Bill Raw':'New-Leasing-Master-ola-Operator Bill Raw','Toll/ Parking for Hisaab':'New-Leasing-Master-ola-Toll/ Parking for Hisaab'}, axis=1, inplace=True)
# dfs = ola_df.groupby(pd.Grouper(freq='W', key='Date'))['New-Leasing-Master-ola-Operator Bill Raw','New-Leasing-Master-ola-Toll/ Parking for Hisaab'].sum()
# dfs.index -= pd.Timedelta(days=6)
# ola_1=pd.DataFrame(dfs)

,New-Leasing-Master-ola-Operator Bill Raw,New-Leasing-Master-ola-Toll/ Parking for Hisaab
Date,,
2021-12-27,4437960.20,263996.0
2022-01-03,3827247.45,240080.0
2022-01-10,3112730.19,182274.0
2022-01-17,3261529.36,197525.0
2022-01-24,3730143.10,231829.0
2022-01-31,4523728.10,275196.0
2022-02-07,5152209.17,325116.0
2022-02-14,5397906.07,335971.0
2022-02-21,5308858.88,340249.0


# mumbai car master and leasings-vinay bhai

In [11]:
#########################################################car master#####################################################

#uber

sheet= client.open_by_key(car_master_vinay_bhai)
ws= sheet.worksheet('Uber')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
uber_df=dfm.iloc[:,[0,15,21,23]]
uber_df['Week Start Date'] = pd.to_datetime(uber_df['Week Start Date'], errors='coerce',format='%d/%m/%Y')
uber_df['Tolls']=uber_df['Tolls'].str.replace(',','')
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].str.replace(',','')
uber_df['Incentive'] = uber_df['Incentive'].str.replace(',','')
uber_df['Tolls'] = pd.to_numeric(uber_df['Tolls'], errors = 'coerce')
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].astype(float)
uber_df['Incentive'] = pd.to_numeric(uber_df['Incentive'], errors = 'coerce')
uber_df.rename({'Week Start Date':'Week start Date','Tolls': 'car-master-Uber-toll', '60%+Tolls + Tips': 'car-master-Uber-60%+Tolls + Tips','Incentive':'car-master-Uber-Incentive'}, axis=1, inplace=True)
uber_chunabhatti = uber_df.groupby(pd.Grouper(freq='W', key='Week start Date'))['car-master-Uber-toll','car-master-Uber-60%+Tolls + Tips','car-master-Uber-Incentive'].sum()
uber_chunabhatti.index -= pd.Timedelta(days=6)

#there is no tab of b2b in vinay bhai car master

#Penalty

sheet= client.open_by_key(car_master_vinay_bhai)
ws= sheet.worksheet('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = pd.to_numeric(Penalty_df['Fine Amount'], errors = 'coerce')
Penalty_df.rename({'Fine Amount': 'car-master-Penalty-Fine Amount'}, axis=1, inplace=True)
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['car-master-Penalty-Fine Amount'].sum()
df.index -= pd.Timedelta(days=6)
Penalty_chunabhatti=pd.DataFrame(df)

#Adjustment

sheet= client.open_by_key(chirag_sir_adjustment_summary)# from chirag sir adjustments to gather where type is not rejected
ws= sheet.worksheet('Adjustment')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,1]]
reject_df=df.loc[df['Rejection'] != 'reject']
unique_type=reject_df.iloc[:,[0]]

sheet= client.open_by_key(car_master_vinay_bhai)#Adjustments - take selected type in selected column
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,3,5]]
vinay_adjustment_df=df[df["Type"].isin(unique_type["Type"])]

dfm=vinay_adjustment_df
dfm['Date'] = dfm['Date'].str.replace('/','-')
dfm['Date'] = pd.to_datetime(dfm['Date'], errors='coerce',format='%d-%m-%Y')
dfm['Amount'] = dfm['Amount'].str.replace(',','')
dfm['Amount'] = pd.to_numeric(dfm['Amount'], errors = 'coerce')
dfm.rename({'Amount': 'car-master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = dfm.groupby(pd.Grouper(freq='W', key='Date'))['car-master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment_chunabhatti=pd.DataFrame(dfs)

#RTO

sheet= client.open_by_key(car_master_vinay_bhai)
ws= sheet.worksheet('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[1,6]]
rto_df['Date in Payout'] = rto_df['Date in Payout'].str.replace('/','-')
rto_df['Date in Payout'] = pd.to_datetime(rto_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
rto_df.rename({'Fine Amount': 'car-master-RTO-Fine Amount'}, axis=1, inplace=True)
dfs = rto_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['car-master-RTO-Fine Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
rto_chunabhatti=pd.DataFrame(dfs)

###############################################New leasing master chunabhatti###########################################

#adjustment

sheet= client.open_by_key(leasing_vinay_bhai)
ws= sheet.worksheet('Adjustment')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
adjustment_df=dfm.iloc[:,[0,10]]
adjustment_df['Date'] = adjustment_df['Date'].str.replace('-','/')
adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d/%m/%Y')
adjustment_df['Amount'] = adjustment_df['Amount'].str.replace(',','')
adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
adjustment_df.rename({'Amount': 'New-Leasing-Master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = adjustment_df.groupby(pd.Grouper(freq='W', key='Date'))['New-Leasing-Master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment_2=pd.DataFrame(dfs)
adjustment_2

#RTO

sheet= client.open_by_key(leasing_vinay_bhai)
ws= sheet.worksheet('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[0,6]]
rto_df['Date of Fine'] = rto_df['Date of Fine'].str.replace('-','/')
rto_df['Date of Fine'] = pd.to_datetime(rto_df['Date of Fine'], errors='coerce',format='%d/%m/%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
rto_df.rename({'Fine Amount': 'New-Leasing-Master-RTO-Fine Amount'}, axis=1, inplace=True)
dfs = rto_df.groupby(pd.Grouper(freq='W', key='Date of Fine'))['New-Leasing-Master-RTO-Fine Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
rto_2=pd.DataFrame(dfs)

#Penalty

sheet= client.open_by_key(leasing_vinay_bhai)
ws= sheet.worksheet('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = pd.to_numeric(Penalty_df['Fine Amount'], errors = 'coerce')
Penalty_df.rename({'Fine Amount': 'New-Leasing-Master-Penalty-Fine Amount'}, axis=1, inplace=True)
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['New-Leasing-Master-Penalty-Fine Amount'].sum()
df.index -= pd.Timedelta(days=6)
Penalty_2=pd.DataFrame(df)

#there is data in ola for leasing master vinay bhai

#weekly sheet

sheet= client.open_by_key(leasing_vinay_bhai)
ws= sheet.worksheet('Weekly Sheet')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
weekly_df=dfm.iloc[:,[0,7,13,18,22]]
weekly_df['Start Date'] = weekly_df['Start Date'].str.replace('-','/')
weekly_df['Start Date'] = pd.to_datetime(weekly_df['Start Date'], errors='coerce',format='%d/%m/%Y')
weekly_df['Rental Amount'] = weekly_df['Rental Amount'].str.replace(',','')
weekly_df['Uber Rev 1'] = weekly_df['Uber Rev 1'].str.replace(',','')
weekly_df['Uber Rev 2'] = weekly_df['Uber Rev 2'].str.replace(',','')
weekly_df['Uber Incentive'] = weekly_df['Uber Incentive'].str.replace(',','')
weekly_df['Rental Amount'] = pd.to_numeric(weekly_df['Rental Amount'], errors = 'coerce')
weekly_df['Uber Rev 1'] = pd.to_numeric(weekly_df['Uber Rev 1'], errors = 'coerce')
weekly_df['Uber Rev 2'] = pd.to_numeric(weekly_df['Uber Rev 2'], errors = 'coerce')
weekly_df['Uber Incentive'] = pd.to_numeric(weekly_df['Uber Incentive'], errors = 'coerce')
weekly_df.rename({'Rental Amount':'New-Leasing-Master-weekly-Rental Amount','Uber Rev 1':'New-Leasing-Master-weekly-Uber Rev 1','Uber Rev 2':'New-Leasing-Master-weekly-Uber Rev 2','Uber Incentive':'New-Leasing-Master-weekly-Uber Incentive'}, axis=1, inplace=True)
weekly = weekly_df.groupby(pd.Grouper(freq='W', key='Start Date'))['New-Leasing-Master-weekly-Rental Amount','New-Leasing-Master-weekly-Uber Rev 1','New-Leasing-Master-weekly-Uber Rev 2','New-Leasing-Master-weekly-Uber Incentive'].sum()
weekly.index -= pd.Timedelta(days=6)
weekly_2=pd.DataFrame(weekly)

#summary
summary=pd.concat([week_end_date,city,uber_chunabhatti,Penalty_chunabhatti,adjustment_chunabhatti,rto_chunabhatti,adjustment_2,rto_2,Penalty_2,weekly_2], axis=1)
summary.fillna('', inplace=True)
summary.index.name='week start date'
summary_df=summary.reset_index()
vinay_sums=summary_df.sort_values(by='week start date')

#pushing
sheet= clients.open_by_key(chirag_sir_adjustment_summary)
ws= sheet.worksheet_by_title('Mumbai Chunabhatti Summary')
ws.clear()
ws.set_dataframe(vinay_sums,(1,1))
print("Mumbai Chunabhatti Summary updated")

Mumbai Chunabhatti Summary updated


# pune car master 

In [3]:
#uber

sheet= client.open_by_key(pune_car_master)
ws= sheet.worksheet('Uber')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
uber_df=dfm.iloc[:,[0,15,21,23]]
uber_df['Week start Date'] = uber_df['Week start Date'].str.replace('-','/')
uber_df['Week start Date'] = pd.to_datetime(uber_df['Week start Date'], errors='coerce',format='%d/%m/%Y')
uber_df['Tolls']=uber_df['Tolls'].str.replace(',','')
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].str.replace(',','')
uber_df['Incentive'] = uber_df['Incentive'].str.replace(',','')
uber_df['Tolls'] = uber_df['Tolls'].astype(float)
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].astype(float)
uber_df['Incentive'] = uber_df['Incentive'].astype(float)
uber_df.rename({'Tolls': 'car-master-Uber-toll', '60%+Tolls + Tips': 'car-master-Uber-60%+Tolls + Tips','Incentive':'car-master-Uber-Incentive'}, axis=1, inplace=True)
uber_pune = uber_df.groupby(pd.Grouper(freq='W', key='Week start Date'))['car-master-Uber-toll','car-master-Uber-60%+Tolls + Tips','car-master-Uber-Incentive'].sum()
uber_pune.index -= pd.Timedelta(days=6)

#there is no tab of b2b in pune car master

#Penalty

sheet= client.open_by_key(pune_car_master)
ws= sheet.worksheet('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = pd.to_numeric(Penalty_df['Fine Amount'], errors = 'coerce')
Penalty_df.rename({'Fine Amount': 'car-master-Penalty-Fine Amount'}, axis=1, inplace=True)
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['car-master-Penalty-Fine Amount'].sum()
df.index -= pd.Timedelta(days=6)
Penalty_pune=pd.DataFrame(df)

#Adjustment

sheet= client.open_by_key(chirag_sir_adjustment_summary)# from chirag sir adjustments to gather where type is not rejected
ws= sheet.worksheet('Adjustment')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,1]]
reject_df=df.loc[df['Rejection'] != 'reject']
unique_type=reject_df.iloc[:,[0]]

sheet= client.open_by_key(pune_car_master)#Adjustments - take selected type in selected column
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,3,5]]
pune_adjustment_df=df[df["Type"].isin(unique_type["Type"])]

dfm=pune_adjustment_df
dfm['Date'] = dfm['Date'].str.replace('/','-')
dfm['Date'] = pd.to_datetime(dfm['Date'], errors='coerce',format='%d-%m-%Y')
dfm['Amount'] = dfm['Amount'].str.replace(',','')
dfm['Amount'] = pd.to_numeric(dfm['Amount'], errors = 'coerce')
dfm.rename({'Amount': 'car-master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = dfm.groupby(pd.Grouper(freq='W', key='Date'))['car-master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment_pune=pd.DataFrame(dfs)
adjustment_pune

#RTO

sheet= client.open_by_key(pune_car_master)
ws= sheet.worksheet('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[1,6]]
rto_df['Date in Payout'] = rto_df['Date in Payout'].str.replace('/','-')
rto_df['Date in Payout'] = pd.to_datetime(rto_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
rto_df.rename({'Fine Amount': 'car-master-RTO-Fine Amount'}, axis=1, inplace=True)
dfs = rto_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['car-master-RTO-Fine Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
rto_pune=pd.DataFrame(dfs)

#summary
summary=pd.concat([week_end_date,city,uber_pune,Penalty_pune,adjustment_pune,rto_pune], axis=1)
summary.fillna('', inplace=True)
summary.index.name='week start date'
summary_df=summary.reset_index()
pune_sums=summary_df.sort_values(by='week start date')

#pushing
sheet= clients.open_by_key(chirag_sir_adjustment_summary)
ws= sheet.worksheet_by_title('Pune Summary')
ws.clear()
ws.set_dataframe(pune_sums,(1,1))
print("Pune Summary updated")

Pune Summary updated


# Hyderabad car master and leasing

In [4]:
################################################car master############################################################3

#uber

sheet= client.open_by_key(hyd_car_master)
ws= sheet.worksheet('Uber')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
uber_df=dfm.iloc[:,[0,13,16,17,19,21]]
uber_df['Week Start Date'] = uber_df['Week Start Date'].str.replace('-','/')
uber_df['Week Start Date'] = pd.to_datetime(uber_df['Week Start Date'], errors='coerce',format='%d/%m/%Y')
uber_df['Tolls']=uber_df['Tolls'].str.replace(',','')
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].str.replace(',','')
uber_df['Incentive'] = uber_df['Incentive'].str.replace(',','')
uber_df['Net Fare'] = uber_df['Net Fare'].str.replace(',','')
uber_df['Earning w/o Toll'] = uber_df['Earning w/o Toll'].str.replace(',','')
uber_df['Tolls'] = uber_df['Tolls'].astype(float)
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].astype(float)
uber_df['Incentive'] = uber_df['Incentive'].astype(float)
uber_df['Net Fare'] = pd.to_numeric(uber_df['Net Fare'], errors = 'coerce')
uber_df['Earning w/o Toll'] = pd.to_numeric(uber_df['Earning w/o Toll'], errors = 'coerce')
uber_df.rename({'Week Start Date':'Week start Date','Tolls': 'car-master-Uber-toll','Net Fare':'car-master-Uber-Net Fare','Earning w/o Toll':'car-master-Uber-Earning w/o Toll', '60%+Tolls + Tips': 'car-master-Uber-60%+Tolls + Tips','Incentive':'car-master-Uber-Incentive'}, axis=1, inplace=True)
hyd_uber = uber_df.groupby(pd.Grouper(freq='W', key='Week start Date'))['car-master-Uber-toll','car-master-Uber-Net Fare','car-master-Uber-Earning w/o Toll','car-master-Uber-60%+Tolls + Tips','car-master-Uber-Incentive'].sum()
hyd_uber.index -= pd.Timedelta(days=6)

#b2b not in hyderabad car master

#Penalty

sheet= client.open_by_key(hyd_car_master)
ws= sheet.worksheet('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = pd.to_numeric(Penalty_df['Fine Amount'], errors = 'coerce')
Penalty_df.rename({'Fine Amount': 'car-master-Penalty-Fine Amount'}, axis=1, inplace=True)
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['car-master-Penalty-Fine Amount'].sum()
df.index -= pd.Timedelta(days=6)
hyd_penalty=pd.DataFrame(df)

#Adjustment

sheet= client.open_by_key(hyd_car_master)#Adjustments - take selected type in selected column
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,3,5]]
reject_df_hyd=df.loc[df['type'] != 'Deposit Adjustments']
dfm=reject_df_hyd
dfm['Date'] = dfm['Date'].str.replace('-','/')
dfm['Date'] = pd.to_datetime(dfm['Date'], errors='coerce',format='%d/%m/%Y')
dfm['Amount'] = dfm['Amount'].str.replace(',','')
dfm['Amount'] = pd.to_numeric(dfm['Amount'], errors = 'coerce')
dfm.rename({'Amount': 'car-master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = dfm.groupby(pd.Grouper(freq='W', key='Date'))['car-master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment_hyd=pd.DataFrame(dfs)

#RTO

sheet= client.open_by_key(hyd_car_master)
ws= sheet.worksheet('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[0,6]]
rto_df['Date of Fine'] = rto_df['Date of Fine'].str.replace('/','-')
rto_df['Date of Fine'] = pd.to_datetime(rto_df['Date of Fine'], errors='coerce',format='%d-%m-%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
rto_df.rename({'Fine Amount': 'car-master-RTO-Fine Amount'}, axis=1, inplace=True)
dfs = rto_df.groupby(pd.Grouper(freq='W', key='Date of Fine'))['car-master-RTO-Fine Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
rto_hyd=pd.DataFrame(dfs)

###################################################leasing master#####################################################

#Adjustments

sheet= client.open_by_key(hyd_leasing)
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
adjustment_df=dfm.iloc[:,[0,6]]
adjustment_df['Date'] = adjustment_df['Date'].str.replace('-','/')
adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d/%m/%Y')
adjustment_df['Amount'] = adjustment_df['Amount'].str.replace(',','')
adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
adjustment_df.rename({'Amount': 'New-Leasing-Master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = adjustment_df.groupby(pd.Grouper(freq='W', key='Date'))['New-Leasing-Master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
leasing_adjustment_hyd=pd.DataFrame(dfs)

#RTO

sheet= client.open_by_key(hyd_leasing)
ws= sheet.worksheet('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[1,6]]
rto_df['Date of Fine'] = rto_df['Date of Fine'].str.replace('-','/')
rto_df['Date of Fine'] = pd.to_datetime(rto_df['Date of Fine'], errors='coerce',format='%d/%m/%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
rto_df.rename({'Fine Amount': 'New-Leasing-Master-RTO-Fine Amount'}, axis=1, inplace=True)
dfs = rto_df.groupby(pd.Grouper(freq='W', key='Date of Fine'))['New-Leasing-Master-RTO-Fine Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
leasing_rto_hyd=pd.DataFrame(dfs)

#Penalty

sheet= client.open_by_key(hyd_leasing)
ws= sheet.worksheet('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[0,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = pd.to_numeric(Penalty_df['Fine Amount'], errors = 'coerce')
Penalty_df.rename({'Fine Amount': 'New-Leasing-Master-Penalty-Fine Amount'}, axis=1, inplace=True)
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['New-Leasing-Master-Penalty-Fine Amount'].sum()
df.index -= pd.Timedelta(days=6)
leasing_penalty_hyd=pd.DataFrame(df)

# there is no ola for hyderabad leasing

#weekly sheet

sheet= client.open_by_key(hyd_leasing)
ws= sheet.worksheet('Weekly Sheet')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
weekly_df=dfm.iloc[:,[0,5,10,14,17]]
weekly_df['Start Date'] = weekly_df['Start Date'].str.replace('-','/')
weekly_df['Start Date'] = pd.to_datetime(weekly_df['Start Date'], errors='coerce',format='%d/%m/%Y')
weekly_df['Rental Amount'] = weekly_df['Rental Amount'].str.replace(',','')
weekly_df['Uber Rev 1'] = weekly_df['Uber Rev 1'].str.replace(',','')
weekly_df['Uber Rev 2'] = weekly_df['Uber Rev 2'].str.replace(',','')
weekly_df['Uber Incentive'] = weekly_df['Uber Incentive'].str.replace(',','')
weekly_df['Rental Amount'] = pd.to_numeric(weekly_df['Rental Amount'], errors = 'coerce')
weekly_df['Uber Rev 1'] = pd.to_numeric(weekly_df['Uber Rev 1'], errors = 'coerce')
weekly_df['Uber Rev 2'] = pd.to_numeric(weekly_df['Uber Rev 2'], errors = 'coerce')
weekly_df['Uber Incentive'] = pd.to_numeric(weekly_df['Uber Incentive'], errors = 'coerce')
weekly_df.rename({'Rental Amount':'New-Leasing-Master-weekly-Rental Amount','Uber Rev 1':'New-Leasing-Master-weekly-Uber Rev 1','Uber Rev 2':'New-Leasing-Master-weekly-Uber Rev 2','Uber Incentive':'New-Leasing-Master-weekly-Uber Incentive'}, axis=1, inplace=True)
weekly = weekly_df.groupby(pd.Grouper(freq='W', key='Start Date'))['New-Leasing-Master-weekly-Rental Amount','New-Leasing-Master-weekly-Uber Rev 1','New-Leasing-Master-weekly-Uber Rev 2','New-Leasing-Master-weekly-Uber Incentive'].sum()
weekly.index -= pd.Timedelta(days=6)
leasing_weekly_hyd=pd.DataFrame(weekly)

#summary
summary=pd.concat([week_end_date,city,hyd_uber,hyd_penalty,adjustment_hyd,rto_hyd,leasing_adjustment_hyd,leasing_rto_hyd,leasing_penalty_hyd,leasing_weekly_hyd], axis=1)
summary.fillna('', inplace=True)
summary.index.name='week start date'
summary_df=summary.reset_index()
hyd_sums=summary_df.sort_values(by='week start date')

#pushing
sheet= clients.open_by_key(chirag_sir_adjustment_summary)
ws= sheet.worksheet_by_title('Hyderabad Summary')
ws.clear()
ws.set_dataframe(hyd_sums,(1,1))
print("Hyderabad Summary updated")

Hyderabad Summary updated


# Bangalore car master and leasing

In [5]:
#wow_driver_payment

sheet= client.open_by_key(bangalore)
ws= sheet.worksheet('WOW_Driver Payments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
wow_df=dfm.iloc[:,[1,6,8,13,14]]
wow_df['Week Begin'] = wow_df['Week Begin'].str.replace('-','/')
wow_df['Week Begin'] = pd.to_datetime(wow_df['Week Begin'], errors='coerce',format='%d/%m/%Y')
wow_df['Toll']=wow_df['Toll'].str.replace(',','')
wow_df['Net Earnings(Toll-Tip)']=wow_df['Net Earnings(Toll-Tip)'].str.replace(',','')
wow_df['Incentives']=wow_df['Incentives'].str.replace(',','')
wow_df['Driver Pay']=wow_df['Driver Pay'].str.replace(',','')
wow_df['Toll'] = pd.to_numeric(wow_df['Toll'], errors = 'coerce')
wow_df['Net Earnings(Toll-Tip)'] = pd.to_numeric(wow_df['Net Earnings(Toll-Tip)'], errors = 'coerce')
wow_df['Incentives'] = pd.to_numeric(wow_df['Incentives'], errors = 'coerce')
wow_df['Driver Pay'] = pd.to_numeric(wow_df['Driver Pay'], errors = 'coerce')
wow_df.rename({'Week Begin':'Week start Date','Toll':'car-master-Uber-toll','Net Earnings(Toll-Tip)':'car-master-Uber-Net Earnings(Toll-Tip)','Incentives':'car-master-Uber-Incentive','Driver Pay':'car-master-Uber-Driver Pay'}, axis=1, inplace=True)
wow_bangalore = wow_df.groupby(pd.Grouper(freq='W', key='Week start Date'))['car-master-Uber-toll','car-master-Uber-Net Earnings(Toll-Tip)','car-master-Uber-Incentive','car-master-Uber-Driver Pay'].sum()
wow_bangalore.index -= pd.Timedelta(days=6)

#rental

sheet= client.open_by_key(bangalore)
ws= sheet.worksheet('Rentals')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rental_df=dfm.iloc[:,[1,5,10,13,14,15]]
rental_df['Week Begin'] = rental_df['Week Begin'].str.replace('-','/')
rental_df['Week Begin'] = pd.to_datetime(rental_df['Week Begin'], errors='coerce',format='%d/%m/%Y')
rental_df['Earnings']=rental_df['Earnings'].str.replace(',','')
rental_df['Net Earnings']=rental_df['Net Earnings'].str.replace(',','')
rental_df['Everest Rental']=rental_df['Everest Rental'].str.replace(',','')
rental_df['Driver Incentives']=rental_df['Driver Incentives'].str.replace(',','')
rental_df['Driver Rental']=rental_df['Driver Rental'].str.replace(',','')
rental_df['Earnings'] = pd.to_numeric(rental_df['Earnings'], errors = 'coerce')
rental_df['Net Earnings'] = pd.to_numeric(rental_df['Net Earnings'], errors = 'coerce')
rental_df['Everest Rental'] = pd.to_numeric(rental_df['Everest Rental'], errors = 'coerce')
rental_df['Driver Incentives'] = pd.to_numeric(rental_df['Driver Incentives'], errors = 'coerce')
rental_df['Driver Rental'] = pd.to_numeric(rental_df['Driver Rental'], errors = 'coerce')
rental_df.rename({'Week Begin':'Week start Date','Earnings':'New-Leasing-Master-Earnings','Net Earnings':'New-Leasing-Master-Net Earnings','Everest Rental':'New-Leasing-Master-Everest Rental','Driver Incentives':'New-Leasing-Master-Driver Incentives','Driver Rental':'New-Leasing-Master-Driver Rental'}, axis=1, inplace=True)
rental_bangalore = rental_df.groupby(pd.Grouper(freq='W', key='Week start Date'))['New-Leasing-Master-Earnings','New-Leasing-Master-Net Earnings','New-Leasing-Master-Everest Rental','New-Leasing-Master-Driver Incentives','New-Leasing-Master-Driver Rental'].sum()
rental_bangalore.index -= pd.Timedelta(days=6)

#summary
summary=pd.concat([week_end_date,city,wow_bangalore,rental_bangalore], axis=1)
summary.fillna('', inplace=True)
summary.index.name='week start date'
summary_df=summary.reset_index()
bangalore_sums=summary_df.sort_values(by='week start date')

#pushing
sheet= clients.open_by_key(chirag_sir_adjustment_summary)
ws= sheet.worksheet_by_title('Bangalore Summary')
ws.clear()
ws.set_dataframe(bangalore_sums,(1,1))
print("Bangalore Summary updated")


Bangalore Summary updated


# Delhi car master and leasing

In [6]:
############################################car master###################################################################

# there is no data in uber

# there is no tab of b2b 

#Penalty

sheet= client.open_by_key(delhi)
ws= sheet.worksheet('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] =pd.to_datetime(Penalty_df['Date in Payout'])
Penalty_df['-334500'] = Penalty_df['-334500'].str.replace(',','')
Penalty_df['-334500'] = pd.to_numeric(Penalty_df['-334500'], errors = 'coerce')
Penalty_df.rename({'-334500': 'car-master-Penalty-P-334500'}, axis=1, inplace=True)
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['car-master-Penalty-P-334500'].sum()
df.index -= pd.Timedelta(days=6)
Penalty_delhi=pd.DataFrame(df)

#Adjustment

sheet= client.open_by_key(delhi)
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
df = pd.DataFrame(data,columns=headers)
dfm=df.iloc[:,[0,5]]
dfm['Date'] = dfm['Date'].str.replace('/','-')
dfm['Date'] = pd.to_datetime(dfm['Date'])
dfm['Amount'] = dfm['Amount'].str.replace(',','')
dfm['Amount'] = pd.to_numeric(dfm['Amount'], errors = 'coerce')
dfm.rename({'Amount': 'car-master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = dfm.groupby(pd.Grouper(freq='W', key='Date'))['car-master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment_delhi=pd.DataFrame(dfs)

#Incentive

sheet= client.open_by_key(delhi)
ws= sheet.worksheet('Incentive')
data = ws.get_all_values()
headers = data.pop(0)
df = pd.DataFrame(data,columns=headers)
dfm=df.iloc[:,[0,5]]
dfm['Date'] = dfm['Date'].str.replace('/','-')
dfm['Date'] = pd.to_datetime(dfm['Date'])
dfm['Amount'] = dfm['Amount'].str.replace(',','')
dfm['Amount'] = pd.to_numeric(dfm['Amount'], errors = 'coerce')
dfm.rename({'Amount': 'car-master-Incentive-Amount'}, axis=1, inplace=True)
dfs = dfm.groupby(pd.Grouper(freq='W', key='Date'))['car-master-Incentive-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
Incentive_delhi=pd.DataFrame(dfs)

#there is no tab for rto

###################################################leasing#############################################################

#leasing

sheet= client.open_by_key(delhi)
ws= sheet.worksheet('Leasing')
data = ws.get_all_values()
headers = data.pop(0)
df = pd.DataFrame(data,columns=headers)
dfm=df.iloc[:,[0,3,4,7,9,10]]
dfm['Date'] = dfm['Date'].str.replace('/','-')
dfm['Date'] = pd.to_datetime(dfm['Date'])
dfm['Lease Rent'] = dfm['Lease Rent'].str.replace(',','')
dfm['Lease Rent'] = pd.to_numeric(dfm['Lease Rent'], errors = 'coerce')
dfm['Net Fares'] = dfm['Net Fares'].str.replace(',','')
dfm['Net Fares'] = pd.to_numeric(dfm['Net Fares'], errors = 'coerce')
dfm['Toll'] = dfm['Toll'].str.replace(',','')
dfm['Toll'] = pd.to_numeric(dfm['Toll'], errors = 'coerce')
dfm['Tips/Refund'] = dfm['Tips/Refund'].str.replace(',','')
dfm['Tips/Refund'] = pd.to_numeric(dfm['Tips/Refund'], errors = 'coerce')
dfm['UP tax'] = dfm['UP tax'].str.replace(',','')
dfm['UP tax'] = pd.to_numeric(dfm['UP tax'], errors = 'coerce')
dfm.rename({'Lease Rent': 'New-Leasing-Master-Lease Rent','Net Fares':'New-Leasing-Master-Net Fares','Toll':'New-Leasing-Master-Toll','Tips/Refund':'New-Leasing-Master-Tips/Refund','UP tax':'New-Leasing-Master-UP tax'}, axis=1, inplace=True)
dfs = dfm.groupby(pd.Grouper(freq='W', key='Date'))['New-Leasing-Master-Lease Rent','New-Leasing-Master-Net Fares','New-Leasing-Master-Toll','New-Leasing-Master-Tips/Refund','New-Leasing-Master-UP tax'].sum()
dfs.index -= pd.Timedelta(days=6)
leasing_delhi=pd.DataFrame(dfs)

#summary
summary=pd.concat([week_end_date,city,Penalty_delhi,adjustment_delhi,Incentive_delhi,leasing_delhi], axis=1)
summary.fillna('', inplace=True)
summary.index.name='week start date'
summary_df=summary.reset_index()
delhi_sums=summary_df.sort_values(by='week start date')

#pushing
sheet= clients.open_by_key(chirag_sir_adjustment_summary)
ws= sheet.worksheet_by_title('Delhi Summary')
ws.clear()
ws.set_dataframe(delhi_sums,(1,1))
print("Delhi Summary updated")

Delhi Summary updated


In [41]:
sums.sample()

,week start date,week end date,City,car-master-Uber-toll,car-master-Uber-60%+Tolls + Tips,car-master-Uber-Incentive,car-master-B2B-Fuel,car-master-B2B-Toll,car-master-B2B-TotalAmount,car-master-Penalty-Fine Amount,...,car-master-RTO-Fine Amount,New-Leasing-Master-Adjustment-Amount,New-Leasing-Master-RTO-Fine Amount,New-Leasing-Master-Penalty-Fine Amount,New-Leasing-Master-ola-Operator Bill Raw,New-Leasing-Master-ola-Toll/ Parking for Hisaab,New-Leasing-Master-weekly-Rental Amount,New-Leasing-Master-weekly-Uber Rev 1,New-Leasing-Master-weekly-Uber Rev 2,New-Leasing-Master-weekly-Uber Incentive
4,2022-01-24,,,628302.0,5349606.0,508900.0,40134.85,10086.0,103620.85,-73750.0,...,-79794.0,75403.0,-146634.0,-12557.0,3730143.1,231829.0,1816200.0,1732745.1,730546.48,134500.0


In [44]:
vinay_sums.sample()

,week start date,week end date,City,car-master-Uber-toll,car-master-Uber-60%+Tolls + Tips,car-master-Uber-Incentive,car-master-Penalty-Fine Amount,car-master-Adjustment-Amount,car-master-RTO-Fine Amount,New-Leasing-Master-Adjustment-Amount,New-Leasing-Master-RTO-Fine Amount,New-Leasing-Master-Penalty-Fine Amount,New-Leasing-Master-weekly-Rental Amount,New-Leasing-Master-weekly-Uber Rev 1,New-Leasing-Master-weekly-Uber Rev 2,New-Leasing-Master-weekly-Uber Incentive
11,2022-06-06,,,19115.0,543768.0,24900.0,-2800.0,1825.0,0.0,21540.0,-293105.0,-2000.0,642650.0,620087.36,407849.32,25400.0


In [45]:
pune_sums.sample()

,week start date,week end date,City,car-master-Uber-toll,car-master-Uber-60%+Tolls + Tips,car-master-Uber-Incentive,car-master-Penalty-Fine Amount,car-master-Adjustment-Amount,car-master-RTO-Fine Amount
8,2022-03-28,,,823.0,21371.0,2350.0,0.0,0.0,0.0


In [26]:
# dfs=[sums,pune_sums]
# dataframe=pd.DataFrame()
# for i in dfs:
#     print(dataframe.append(i))
# print(dataframe)


   week start date week end date City car-master-Uber-toll  \
26      2020-01-13                                           
27      2020-01-20                                           
28      2020-01-27                                           
29      2020-02-03                                           
30      2020-02-10                                           
..             ...           ...  ...                  ...   
21      2022-05-23                                388844.0   
22      2022-05-30                                389451.0   
23      2022-06-06                                338569.0   
24      2022-06-13                                320259.0   
25      2022-06-20                                           

   car-master-Uber-60%+Tolls + Tips car-master-Uber-Incentive  \
26                                                              
27                                                              
28                                                          

In [75]:
#Adjustments (unique Type ------ chirag sir sheet )

# sheet= client.open_by_key(car_master)
# ws= sheet.worksheet('Adjustments')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# adjustment_df=dfm.iloc[:,[0,3,5]]
# array=adjustment_df['Type'].unique()
# df=pd.DataFrame(array,columns=['Type'])
# sheet= client.open_by_key(chirag_sir_adjustment_summary)
# ws= sheet.worksheet('Adjustment')
# ws.update([df.columns.values.tolist()] + df.values.tolist(),value_input_option='USER_ENTERED')

#from chirag sir adjustments to gather where type is not rejected

# sheet= client.open_by_key(chirag_sir_adjustment_summary)
# ws= sheet.worksheet('Adjustment')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# df=dfm.iloc[:,[0,1]]
# reject_df=df.loc[df['Rejection'] != 'reject']
# unique_type=reject_df.iloc[:,[0]]
# unique_type


Summary updated


,Adjustment-Amount
Date,
2021-12-27,23512.00
2022-01-03,15904.00
2022-01-10,34521.00
2022-01-17,18823.00
2022-01-24,34443.00
2022-01-31,33132.00
2022-02-07,39404.00
2022-02-14,26230.00
2022-02-21,50191.00


In [21]:
# #Adjustments

# sheet= client.open_by_key(leasing_master)
# ws= sheet.worksheet('Adjustment')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# adjustment_df=dfm.iloc[:,[0,10]]
# adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d/%m/%Y')
# adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
# dfs = adjustment_df.groupby(pd.Grouper(freq='W', key='Date'))['Amount'].sum()
# adjustment_1=pd.DataFrame(dfs)
# adjustment_1.rename({'Amount': 'Adjustment-Amount'}, axis=1, inplace=True)
# adjustment_1.index -= pd.Timedelta(days=6)
# adjustment_1

,Date,Type,Amount
27,28-12-2021,Other Addition,538
30,29-12-2021,Leasing Adjustments,200
31,29-12-2021,Other Addition,667
32,29-12-2021,Referral Bonus,500
33,29-12-2021,Puncture Bill,130
...,...,...,...
13536,,,
13537,,,
13538,,,
13539,,,


,a,b,City
0,1,1,Mumbai
1,2,2,Mumbai


In [81]:
# #Adjustments

# sheet= client.open_by_key(chirag_sir_adjustment_summary)
# ws= sheet.worksheet('test')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# dfm['Date'] = dfm['Date'].str.replace('/','-')
# dfm['Date'] = pd.to_datetime(dfm['Date'], errors='coerce',format='%d-%m-%Y')
# dfm['Amount'] = pd.to_numeric(dfm['Amount'], errors = 'coerce')
# dfm.rename({'Amount': 'car-master-Adjustment-Amount'}, axis=1, inplace=True)
# dfs = dfm.groupby(pd.Grouper(freq='W', key='Date'))['car-master-Adjustment-Amount'].sum()
# dfs.index -= pd.Timedelta(days=6)
# adjustment=pd.DataFrame(dfs)



################################################################New-Leasing-Master##########################################

#Adjustments

sheet= client.open_by_key(leasing_master)
ws= sheet.worksheet('Adjustment')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
adjustment_df=dfm.iloc[:,[0,10]]
adjustment_df['Date'] = adjustment_df['Date'].str.replace('-','/')
adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d/%m/%Y')
adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
adjustment_df.rename({'Amount': 'New-Leasing-Master-Adjustment-Amount'}, axis=1, inplace=True)
dfs = adjustment_df.groupby(pd.Grouper(freq='W', key='Date'))['New-Leasing-Master-Adjustment-Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment_1=pd.DataFrame(dfs)

,New-Leasing-Master-Adjustment-Amount
Date,
2020-01-13,14445.00
2020-01-20,0.00
2020-01-27,0.00
2020-02-03,0.00
2020-02-10,0.00
...,...
2022-05-23,82772.00
2022-05-30,66534.16
2022-06-06,71652.00


# Rough work

In [222]:
#Penalty

sheet= client.open_by_key(leasing_master)
ws= sheet.worksheet('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = pd.to_numeric(Penalty_df['Fine Amount'], errors = 'coerce')
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['Fine Amount'].sum()
df.index -= pd.Timedelta(days=6)
Penalty=pd.DataFrame(df)
Penalty.rename({'Fine Amount': 'Penalty-Fine Amount'}, axis=1, inplace=True)
Penalty.index.name='Date in Payout'
Penalty_1=Penalty.reset_index()
Penalty_1

,Date in Payout,Penalty-Fine Amount
0,2021-10-11,-200.0
1,2021-10-18,0.0
2,2021-10-25,0.0
3,2021-11-01,0.0
4,2021-11-08,0.0
5,2021-11-15,0.0
6,2021-11-22,0.0
7,2021-11-29,0.0
8,2021-12-06,0.0
9,2021-12-13,0.0


In [227]:
#weekly sheet

sheet= client.open_by_key(leasing_master)
ws= sheet.worksheet('Weekly Sheet')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
weekly_df=dfm.iloc[:,[0,7,13,18,22]]
weekly_df['Start Date'] = weekly_df['Start Date'].str.replace('-','/')
weekly_df['Start Date'] = pd.to_datetime(weekly_df['Start Date'], errors='coerce',format='%d/%m/%Y')
weekly_df['Rental Amount'] = weekly_df['Rental Amount'].str.replace(',','')
weekly_df['Uber Rev 1'] = weekly_df['Uber Rev 1'].str.replace(',','')
weekly_df['Uber Rev 2'] = weekly_df['Uber Rev 2'].str.replace(',','')
weekly_df['Uber Incentive'] = weekly_df['Uber Incentive'].str.replace(',','')
weekly_df['Rental Amount'] = pd.to_numeric(weekly_df['Rental Amount'], errors = 'coerce')
weekly_df['Uber Rev 1'] = pd.to_numeric(weekly_df['Uber Rev 1'], errors = 'coerce')
weekly_df['Uber Rev 2'] = pd.to_numeric(weekly_df['Uber Rev 2'], errors = 'coerce')
weekly_df['Uber Incentive'] = pd.to_numeric(weekly_df['Uber Incentive'], errors = 'coerce')
weekly = weekly_df.groupby(pd.Grouper(freq='W', key='Start Date'))['Rental Amount','Uber Rev 1','Uber Rev 2','Uber Incentive'].sum()
weekly.index -= pd.Timedelta(days=6)
weekly_dfs=pd.DataFrame(weekly)
weekly_dfs.rename({'Rental Amount':'weekly-Rental Amount','Uber Rev 1':'weekly-Uber Rev 1','Uber Rev 2':'weekly-Uber Rev 2','Uber Incentive':'weekly-Uber Incentive'}, axis=1, inplace=True)
weekly_dfs.index.name='Start Date'
weekly_1=weekly_dfs.reset_index()

,Start Date,weekly-Rental Amount,weekly-Uber Rev 1,weekly-Uber Rev 2,weekly-Uber Incentive
0,2021-12-27,2996450.0,2686202.67,1415702.43,283200
1,2022-01-03,2137250.0,1434797.60,754314.33,68950
2,2022-01-10,1779950.0,1191547.40,576257.29,92880
3,2022-01-17,1690800.0,1494674.11,724556.38,180400
4,2022-01-24,1816200.0,1732745.10,730546.48,134500
5,2022-01-31,1994550.0,1867991.23,844314.78,150700
6,2022-02-07,2551350.0,2325065.21,970973.38,211800
7,2022-02-14,2614600.0,2548781.36,1129541.76,246900
8,2022-02-21,2924900.0,2347897.79,1141419.92,237500
9,2022-02-28,2994000.0,2435941.39,1112138.39,257800


In [105]:
# summary=pd.concat([week_end_date,city,uber_column,uber,b2b_column,b2b,Penalty,adjustment_column,adjustment,rto_column,rto], axis=1)
# summary.fillna('', inplace=True)
# summary.index.name='week start date'
# summary_df=summary.reset_index()
# clients = pygsheets.authorize(service_file='client_secret.json')
# sheet= clients.open_by_key(chirag_sir_adjustment_summary)
# ws= sheet.worksheet_by_title('Summary')
# ws.set_dataframe(summary_df,(1,1))

,week start date,week end date,City,Uber,Tolls,60%+Tolls + Tips,Incentive,B2B,Fuel,Toll,TotalAmount,Fine Amount,Adjustment,Amount,RTO,Fine Amount
0,2021-12-27,,,,718333.0,6312626.0,733000.0,,6131.25,2090.0,15271.25,-2035,,,,
1,2022-01-03,,,,440028.0,3748232.0,115200.0,,18905.25,5815.0,45870.25,-39400,,,,-93572.0
2,2022-01-10,,,,400369.0,3500378.0,303300.0,,26473.75,7323.0,64396.75,-15950,,,,-53900.0
3,2022-01-17,,,,520466.0,4458335.0,620250.0,,31745.90,8046.0,73691.90,-34150,,,,-91692.0
4,2022-01-24,,,,628302.0,5349606.0,508900.0,,44565.30,11845.0,105810.30,-28595,,,,-94773.0
5,2022-01-31,,,,683450.0,6215469.0,590300.0,,28408.85,7758.0,80066.85,-78100,,3743.0,,-79794.0
6,2022-02-07,,,,785345.0,7173372.0,607500.0,,29162.55,9418.0,70600.55,-47250,,5257.0,,-112340.0
7,2022-02-14,,,,846039.0,8132669.0,678800.0,,38436.75,10660.0,103146.75,-43850,,4909.0,,-126567.0
8,2022-02-21,,,,871148.0,8120473.0,709300.0,,13256.45,4404.0,37410.45,-71750,,5287.0,,-123424.0
9,2022-02-28,,,,450180.0,7793567.0,745650.0,,20925.25,5438.0,53831.25,-73200,,8611.0,,-105172.0


In [193]:
#Adjustments

leasing_master='1D0U02U0QN7pXsgM4AdOUiUKB4rB-SoTwSspWkuwl_8E'


sheet= client.open_by_key(leasing_master)
ws= sheet.worksheet('Adjustment')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
adjustment_df=dfm.iloc[:,[0,10]]
adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d/%m/%Y')
adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
dfs = adjustment_df.groupby(pd.Grouper(freq='W', key='Date'))['Amount'].sum()
adjustment_1=pd.DataFrame(dfs)
adjustment_1.rename({'Amount': 'Adjustment-Amount'}, axis=1, inplace=True)
adjustment_1.index -= pd.Timedelta(days=6)
adjustment_1
# dfs = adjustment_df['Date'].groupby(adjustment_df.datetime.to_period("W"))['Amount'].sum()
# adjustment_df.groupby(df['Date'].('W-SAT'))
# .date.dt.to_period("W")
# df['date'].groupby(df.date.dt.to_period("W")).
# adjustment_1=pd.DataFrame(dfs)
# adjustment_1
# adjustment_1.rename({'Amount': 'Adjustment-Amount'}, axis=1, inplace=True)
# adjustment_1

,Adjustment-Amount
Date,
2020-01-13,14445.00
2020-01-20,0.00
2020-01-27,0.00
2020-02-03,0.00
2020-02-10,0.00
...,...
2022-05-16,72928.00
2022-05-23,82772.00
2022-05-30,66534.16


In [203]:
#Adjustments

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,3,5]]
adjustment_df=df[df["Type"].isin(unique_type["Type"])]
adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d-%m-%Y')
adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
dfs = adjustment_df.groupby(pd.Grouper(freq='W', key='Date'))['Amount'].sum()
dfs.index -= pd.Timedelta(days=6)
adjustment=pd.DataFrame(dfs)
adjustment.rename({'Amount': 'Adjustment-Amount'}, axis=1, inplace=True)
adjustment

,Adjustment-Amount
Date,
2021-12-27,23512.00
2022-01-03,15904.00
2022-01-10,34521.00
2022-01-17,18823.00
2022-01-24,34443.00
2022-01-31,33132.00
2022-02-07,39404.00
2022-02-14,26230.00
2022-02-21,50191.00


In [147]:
a=uber_df[(uber_df['Week start Date'] == '2022-04-18')]
a

,Week start Date,Tolls,60%+Tolls + Tips,Incentive
16461,2022-04-18,0.0,963.0,0.0
16462,2022-04-18,85.0,7394.0,600.0
16463,2022-04-18,920.0,12134.0,2050.0
16464,2022-04-18,580.0,11442.0,1500.0
16465,2022-04-18,665.0,10668.0,1250.0
...,...,...,...,...
17691,2022-04-18,375.0,500.0,0.0
17692,2022-04-18,583.0,583.0,0.0
17693,2022-04-18,413.0,438.0,850.0
17694,2022-04-18,293.0,293.0,0.0


In [145]:
a['60%+Tolls + Tips'].sum()

9935306.0

In [160]:
#uber

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('Uber')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
uber_df=dfm.iloc[:,[0,15,21,23]]
uber_df['Week start Date'] = pd.to_datetime(uber_df['Week start Date'], errors='coerce',format='%d/%m/%Y')
uber_df['Tolls']=uber_df['Tolls'].str.replace(',','')
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].str.replace(',','')
uber_df['Incentive'] = uber_df['Incentive'].str.replace(',','')
uber_df['Tolls'] = uber_df['Tolls'].astype(float)
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].astype(float)
uber_df['Incentive'] = uber_df['Incentive'].astype(float)
uber = uber_df.groupby(pd.Grouper(freq='W-MON', key='Week start Date'))['Tolls','60%+Tolls + Tips','Incentive'].sum()
uber

,Tolls,60%+Tolls + Tips,Incentive
Week start Date,,,
2021-12-27,718333.0,6312626.0,733000.0
2022-01-03,440028.0,3748232.0,115200.0
2022-01-10,400369.0,3500378.0,303300.0
2022-01-17,520466.0,4458335.0,620250.0
2022-01-24,628302.0,5349606.0,508900.0
2022-01-31,683450.0,6215469.0,590300.0
2022-02-07,785345.0,7173372.0,607500.0
2022-02-14,846039.0,8132669.0,678800.0
2022-02-21,871148.0,8120473.0,709300.0


In [146]:
n=[-7,8,9,1.146]
a=sum(n)
a

11.146

# rough work

In [38]:
week_end_date=pd.DataFrame()
week_end_date['week end date']=''
city=pd.DataFrame()
city['City']=''
uber_column=pd.DataFrame()
uber_column['Uber']=''
b2b_column=pd.DataFrame()
b2b_column['B2B']=''
penalty_column=pd.DataFrame()
penalty_column['Penalty']=''
adjustment_column=pd.DataFrame()
adjustment_column['Adjustment']=''
rto_column=pd.DataFrame()
rto_column['RTO']=''

In [10]:
#Adjustments for comparing

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
adjustment_df=dfm.iloc[:,[0,3,5]]
array=adjustment_df['Type'].unique()
df=pd.DataFrame(array,columns=['Type'])
chirag_sir_adjustment_summary='14bR0MmfLvQi1lyzpIgFn02Qizr9bLwIDqhBu1vqxk2E'
sheet= client.open_by_key(chirag_sir_adjustment_summary)
ws= sheet.worksheet('Adjustment')
ws.batch_clear(['A:A500'])
ws.update([df.columns.values.tolist()] + df.values.tolist(),value_input_option='USER_ENTERED')

{'spreadsheetId': '14bR0MmfLvQi1lyzpIgFn02Qizr9bLwIDqhBu1vqxk2E',
 'updatedRange': 'Adjustment!A1:A167',
 'updatedRows': 167,
 'updatedColumns': 1,
 'updatedCells': 167}

In [12]:
#from chirag sir adjustments to gather where type is not rejected

sheet= client.open_by_key(chirag_sir_adjustment_summary)
ws= sheet.worksheet('Adjustment')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,1]]
reject_df=df.loc[df['Rejection'] != 'reject']
unique_type=reject_df.iloc[:,[0]]
sheet= client.open_by_key(car_master)
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,3,5]]
adjustment_df=df[df["Type"].isin(unique_type["Type"])]
# adjustment_df
# adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d-%m-%Y')
# adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
# dfs = adjustment_df.groupby(pd.Grouper(freq='W', key='Date'))['Amount'].sum()
# dfs.index -= pd.Timedelta(days=6)
# dfs
# adjustment=pd.DataFrame(dfs)
# adjustment.rename({'Amount': 'Adjustment-Amount'}, axis=1, inplace=True)
# adjustment

Date
2021-12-27     23512.00
2022-01-03     15904.00
2022-01-10     34521.00
2022-01-17     18823.00
2022-01-24     34443.00
2022-01-31     33132.00
2022-02-07     39404.00
2022-02-14     26230.00
2022-02-21     50191.00
2022-02-28     51010.00
2022-03-07    226303.00
2022-03-14    154113.04
2022-03-21    118612.00
2022-03-28    132649.99
2022-04-04    106013.00
2022-04-11    164332.00
2022-04-18     81533.20
2022-04-25      7806.91
2022-05-02     58770.00
2022-05-09     74730.00
2022-05-16    139496.00
2022-05-23     67110.00
2022-05-30    -46515.00
2022-06-06     -2539.00
2022-06-13   -173039.59
Name: Amount, dtype: float64

In [5]:
unique_type

,Type
1,Other Addition
2,Leasing Adjustments
3,Referral Bonus
4,Puncture Bill
5,Penalty Reversal
...,...
161,Uber Refund
162,Pending Joining Fee Reversal
163,
164,Reversal of pending joining fee


In [79]:
#uber

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('Uber')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
uber_df=dfm.iloc[:,[0,15,21,23]]
uber_df
uber_df['Week start Date'] = pd.to_datetime(uber_df['Week start Date'], errors='coerce',format='%d/%m/%Y')
uber_df['Tolls']=uber_df['Tolls'].str.replace(',','')
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].str.replace(',','')
uber_df['Incentive'] = uber_df['Incentive'].str.replace(',','')
uber_df['Tolls'] = uber_df['Tolls'].astype('int')
uber_df['60%+Tolls + Tips'] = uber_df['60%+Tolls + Tips'].astype('int')
uber_df['Incentive'] = uber_df['Incentive'].astype('int')
uber = uber_df.groupby(pd.Grouper(freq='W-MON', key='Week start Date'))['Tolls','60%+Tolls + Tips','Incentive'].sum()
uber

,Tolls,60%+Tolls + Tips,Incentive
Week start Date,,,
2021-12-27,718333,6312626,733000
2022-01-03,440028,3748232,115200
2022-01-10,400369,3500378,303300
2022-01-17,520466,4458335,620250
2022-01-24,628302,5349606,508900
2022-01-31,683450,6215469,590300
2022-02-07,785345,7173372,607500
2022-02-14,846039,8132669,678800
2022-02-21,871148,8120473,709300


In [166]:
#Adjustments

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,3,5]]
adjustment_df=df[df["Type"].isin(unique_type["Type"])]
adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d-%m-%Y')
adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
dfs = adjustment_df.groupby(pd.Grouper(freq='W-MON', key='Date'))['Amount'].sum()
adjustment=pd.DataFrame(dfs)
adjustment.rename({'Amount': 'Adjustment-Amount'}, axis=1, inplace=True)
# adjustment

#RTO

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[1,6]]
rto_df['Date in Payout'] = rto_df['Date in Payout'].str.replace('/','-')
rto_df['Date in Payout'] = pd.to_datetime(rto_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
dfs = rto_df.groupby(pd.Grouper(freq='W-MON', key='Date in Payout'))['Fine Amount'].sum()
rto=pd.DataFrame(dfs)
rto.rename({'Fine Amount': 'RTO-Fine Amount'}, axis=1, inplace=True)


,RTO-Fine Amount
Date in Payout,
2022-01-03,-93572.0
2022-01-10,-53900.0
2022-01-17,-91692.0
2022-01-24,-94773.0
2022-01-31,-79794.0
2022-02-07,-112340.0
2022-02-14,-126567.0
2022-02-21,-123424.0
2022-02-28,-105172.0


In [155]:
a=b2b_df[(b2b_df['Date'] >= '2021-12-27') & (df['Date'] <= '2022-01-02')]
a

,Date,Fuel,Toll,TotalAmount
158,2022-01-28,1117.50,486.0,2453.50
159,2022-01-28,0.00,0.0,850.00
160,2022-01-28,0.00,0.0,850.00
161,2022-01-28,544.00,80.0,1124.00
162,2022-01-28,380.80,0.0,880.80
...,...,...,...,...
614,2022-05-31,0.00,0.0,450.00
615,2022-05-31,0.00,0.0,450.00
616,2022-05-31,0.00,0.0,1000.00
617,2022-06-01,1620.00,540.0,3160.00


In [156]:
a['Fuel'].sum()

482272.5

In [2]:
#importing libraries

import pandas as pd
import datetime
from datetime import date, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import warnings
warnings.filterwarnings("ignore")

#creating connection

scope= ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials= ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope) 
client= gspread.authorize(credentials)

#sheet keys

car_master='1_r5OMN1P8Tof5IRaE5jYd-jHaP8j15avQVMycDJdhec'
chirag_sir_adjustment_summary='14bR0MmfLvQi1lyzpIgFn02Qizr9bLwIDqhBu1vqxk2E'

In [158]:
13.3290+12.09+1.90

27.319

In [192]:
#Penalty

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('Penalty')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
Penalty_df=dfm.iloc[:,[1,7]]
Penalty_df['Date in Payout'] = Penalty_df['Date in Payout'].str.replace('/','-')
Penalty_df['Date in Payout'] = pd.to_datetime(Penalty_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].str.replace(',','')
Penalty_df['Fine Amount'] = Penalty_df['Fine Amount'].astype('int')
df = Penalty_df.groupby(pd.Grouper(freq='W', key='Date in Payout'))['Fine Amount'].sum()
Penalty=pd.DataFrame(df)
Penalty.index -= pd.Timedelta(days=6)
Penalty

,Fine Amount
Date in Payout,
2021-12-27,-41435
2022-01-03,-15950
2022-01-10,-34150
2022-01-17,-28595
2022-01-24,-73750
2022-01-31,-45050
2022-02-07,-49600
2022-02-14,-72550
2022-02-21,-73200


In [21]:
#Adjustments

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('Adjustments')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,3,5]]
adjustment_df=df[df["Type"].isin(unique_type["Type"])]
adjustment_df['Date'] = pd.to_datetime(adjustment_df['Date'], errors='coerce',format='%d-%m-%Y')
adjustment_df['Amount'] = pd.to_numeric(adjustment_df['Amount'], errors = 'coerce')
dfs = adjustment_df.groupby(pd.Grouper(freq='W-MON', key='Date'))['Amount'].sum()
adjustment=pd.DataFrame(dfs)
adjustment

,Amount
Date,
2022-01-31,3743.00
2022-02-07,5257.00
2022-02-14,4909.00
2022-02-21,5287.00
2022-02-28,8611.00
2022-03-07,10821.00
2022-03-14,15031.00
2022-03-21,6800.00
2022-03-28,6627.00


In [69]:
adjustment_df.dtypes

Date      datetime64[ns]
Type              object
Amount            object
dtype: object

In [80]:
# for i in adjustment:
#     print(i)

In [23]:
#RTO

sheet= client.open_by_key(car_master)
ws= sheet.worksheet('RTO')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
rto_df=dfm.iloc[:,[1,6]]
rto_df['Date in Payout'] = rto_df['Date in Payout'].str.replace('/','-')
rto_df['Date in Payout'] = pd.to_datetime(rto_df['Date in Payout'], errors='coerce',format='%d-%m-%Y')
rto_df['Fine Amount'] = rto_df['Fine Amount'].str.replace(',','')
rto_df['Fine Amount'] = pd.to_numeric(rto_df['Fine Amount'], errors = 'coerce')
dfs = rto_df.groupby(pd.Grouper(freq='W-MON', key='Date in Payout'))['Fine Amount'].sum()
rto=pd.DataFrame(dfs)
rto

,Fine Amount
Date in Payout,
2022-01-03,-93572.0
2022-01-10,-53900.0
2022-01-17,-91692.0
2022-01-24,-94773.0
2022-01-31,-79794.0
2022-02-07,-112340.0
2022-02-14,-126567.0
2022-02-21,-123424.0
2022-02-28,-105172.0


In [106]:
#pushing
# sheet= clients.open_by_key(chirag_sir_adjustment_summary)
# ws= sheet.worksheet_by_title('akash sir testing')
# ws.clear(start='A12',end='G100')
# ws.set_dataframe(uber,(12,1))
# print("Summary updated")